![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/onnx/HuggingFace_ONNX_in_Spark_NLP_AlbertForSequenceClassification.ipynb)

## Import ONNX AlbertForSequenceClassification models from HuggingFace 🤗  into Spark NLP 🚀

Let's keep in mind a few things before we start 😊

- ONNX support was introduced in  `Spark NLP 5.0.0`, enabling high performance inference for models.
- `AlbertForSequenceClassification` is only available since in `Spark NLP 5.1.1` and after. So please make sure you have upgraded to the latest Spark NLP release
- You can import ALBERT models trained/fine-tuned for sequence classification via `AlbertForSequenceClassification`. These models are usually under `Text Classification` category and have `albert` in their labels
- Reference: [TFAlbertForSequenceClassification](https://huggingface.co/docs/transformers/model_doc/albert#transformers.TFAlbertForSequenceClassification)
- Some [example models](https://huggingface.co/models?filter=albert&pipeline_tag=text-classification)

## Export and Save HuggingFace model

- Let's install `transformers` package with the `onnx` extension and it's dependencies. You don't need `onnx` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.48.2`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.
- Albert uses SentencePiece, so we will have to install that as well

In [ ]:
!pip install -q --upgrade transformers[onnx]==4.48.2 optimum onnx

- HuggingFace has an extension called Optimum which offers specialized model inference, including ONNX. We can use this to import and export ONNX models with `from_pretrained` and `save_pretrained`.
- We'll use [mohsenfayyaz/albert-base-v2-toxicity](https://huggingface.co/mohsenfayyaz/albert-base-v2-toxicity) model from HuggingFace as an example and load it as a `ORTModelForSequenceClassification`, representing an ONNX model.
- In addition to the ALBERT model, we also need to save the `AlbertTokenizer`. This is the same for every model, these are assets (saved in `/assets`) needed for tokenization inside Spark NLP.

In [2]:
from transformers import AutoTokenizer
from optimum.onnxruntime import ORTModelForSequenceClassification

MODEL_NAME = 'mohsenfayyaz/albert-base-v2-toxicity'
EXPORT_PATH = f"onnx_models/{MODEL_NAME}"

ort_model = ORTModelForSequenceClassification.from_pretrained(MODEL_NAME, export=True)
ort_model.save_pretrained(EXPORT_PATH)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.save_pretrained(EXPORT_PATH)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/871 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/46.8M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/428 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/760k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/245 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/46.7M [00:00<?, ?B/s]

('onnx_models/mohsenfayyaz/albert-base-v2-toxicity/tokenizer_config.json',
 'onnx_models/mohsenfayyaz/albert-base-v2-toxicity/special_tokens_map.json',
 'onnx_models/mohsenfayyaz/albert-base-v2-toxicity/spiece.model',
 'onnx_models/mohsenfayyaz/albert-base-v2-toxicity/added_tokens.json',
 'onnx_models/mohsenfayyaz/albert-base-v2-toxicity/tokenizer.json')

Let's have a look inside these two directories and see what we are dealing with:

In [3]:
!ls -l {EXPORT_PATH}

total 49060
-rw-r--r-- 1 root root      914 Jun  7 05:51 config.json
-rw-r--r-- 1 root root 47180295 Jun  7 05:51 model.onnx
-rw-r--r-- 1 root root      286 Jun  7 05:51 special_tokens_map.json
-rw-r--r-- 1 root root   760289 Jun  7 05:51 spiece.model
-rw-r--r-- 1 root root     2598 Jun  7 05:51 tokenizer_config.json
-rw-r--r-- 1 root root  2275547 Jun  7 05:51 tokenizer.json


- We need to move the `spiece.model` file from the tokenizer into an assets folder, as this is where Spark NLP looks for it when working with models like Albert or other SentencePiece-based tokenizers.
- Additionally, we need to extract the `labels` and their corresponding `ids` from the model's config. This mapping will be saved as `labels.txt` inside the same `assets` folder.

In [4]:
!mkdir {EXPORT_PATH}/assets

labels = ort_model.config.id2label
labels = [value for key, value in sorted(labels.items(), reverse=False)]

with open(EXPORT_PATH + '/assets/labels.txt', 'w') as f:
    f.write('\n'.join(labels))

!mv {EXPORT_PATH}/spiece.model {EXPORT_PATH}/assets

In [6]:
!ls -lR {EXPORT_PATH}

onnx_models/mohsenfayyaz/albert-base-v2-toxicity:
total 48320
drwxr-xr-x 2 root root     4096 Jun  7 05:52 assets
-rw-r--r-- 1 root root      914 Jun  7 05:51 config.json
-rw-r--r-- 1 root root 47180295 Jun  7 05:51 model.onnx
-rw-r--r-- 1 root root      286 Jun  7 05:51 special_tokens_map.json
-rw-r--r-- 1 root root     2598 Jun  7 05:51 tokenizer_config.json
-rw-r--r-- 1 root root  2275547 Jun  7 05:51 tokenizer.json

onnx_models/mohsenfayyaz/albert-base-v2-toxicity/assets:
total 748
-rw-r--r-- 1 root root     15 Jun  7 05:52 labels.txt
-rw-r--r-- 1 root root 760289 Jun  7 05:51 spiece.model


In [5]:
!cat {EXPORT_PATH}/assets/labels.txt

Non-Toxic
Toxic

Voila! We have our `spiece.model` inside assets directory and the extracted labels inside `labels.txt`

## Import and Save AlbertForSequenceClassification in Spark NLP


Let's install and setup Spark NLP in Google Colab. For this example, we'll use specific versions of `pyspark` and `spark-nlp` that we've already tested with this transformer model to make sure everything runs smoothly:

If you prefer to use the latest versions, feel free to run:

`!pip install -q pyspark spark-nlp`

Just keep in mind that newer versions might have some changes, so you may need to tweak your code a bit if anything breaks.

In [7]:
!pip install -q pyspark==3.5.4 spark-nlp==5.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.7/635.7 kB 32.8 MB/s eta 0:00:00


Let's start Spark with Spark NLP included via our simple `start()` function

In [8]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  5.5.3
Apache Spark version:  3.5.4


- Let's use `loadSavedModel` functon in `AlbertForSequenceClassification` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `AlbertForSequenceClassification` in runtime like `setMaxSentenceLength`, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.



In [9]:
from sparknlp.annotator import AlbertForSequenceClassification

sequenceClassifier = AlbertForSequenceClassification\
  .loadSavedModel(EXPORT_PATH, spark)\
  .setInputCols(["document",'token'])\
  .setOutputCol("class")\
  .setCaseSensitive(False)\
  .setMaxSentenceLength(128)

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [10]:
sequenceClassifier.write().overwrite().save("./{}_spark_nlp_onnx".format(MODEL_NAME))

Let's clean up stuff we don't need anymore

In [11]:
!rm -rf {EXPORT_PATH}

Awesome 😎  !

This is your AlbertForSequenceClassification model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [12]:
! ls -l {MODEL_NAME}_spark_nlp_onnx

total 46836
-rw-r--r-- 1 root root 47187629 Jun  7 05:57 albert_classification_onnx
-rw-r--r-- 1 root root   760289 Jun  7 05:57 albert_spp
drwxr-xr-x 3 root root     4096 Jun  7 05:57 fields
drwxr-xr-x 2 root root     4096 Jun  7 05:57 metadata


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny AlbertForSequenceClassification model 😊

In [13]:
sequenceClassifier_loaded = AlbertForSequenceClassification.load("./{}_spark_nlp_onnx".format(MODEL_NAME))\
  .setInputCols(["document",'token'])\
  .setOutputCol("class")

You can see what labels were used to train this model via `getClasses` function:

In [14]:
sequenceClassifier_loaded.getClasses()

['Non-Toxic', 'Toxic']

This is how you can use your loaded classifier model in Spark NLP 🚀 pipeline:

In [ ]:
from sparknlp.base import DocumentAssembler
from sparknlp.annotator import Tokenizer
from pyspark.ml import Pipeline

document_assembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols("document") \
    .setOutputCol("token")

pipeline = Pipeline(stages=[
    document_assembler,
    tokenizer,
    sequenceClassifier_loaded
])

example = spark.createDataFrame([
    ["I love you!"],
    ["I feel lucky to be here."],
    ["I hate her!"]
], ["text"])

result = pipeline.fit(example).transform(example)
result.select("text", "class.result").show(truncate=False)

+--------------------+-----------+
|                text|     result|
+--------------------+-----------+
|         I love you!|[Non-Toxic]|
|I feel lucky to b...|[Non-Toxic]|
|         I hate her!|    [Toxic]|
+--------------------+-----------+



That's it! You can now go wild and use hundreds of `AlbertForSequenceClassification` models from HuggingFace 🤗 in Spark NLP 🚀
